<img style="float: right;" src="Abb/baseflow_txt1.png" alt="baseflow" width="200"/>

# Machine Learning (in der Hydrologie)  
Workshop 5. Juli, 2022

## Problemstellung & Übersicht


### Ziel: Vorhersage der mittleren täglichen Flusswassertemperatur der Aschach



![Übersicht Aschach Einzugsgebiet (Nr. 2)](Abb/catchment.png)
*Übersicht Aschach Einzugsgebiet (Nr. 2)*

![Übersicht Aschach Einzugsgebiet (Nr. 2)](Abb/catchment_info.png)
*Einzugsgebiet Eigenschaften*

### To do:

Beantworte folgende Fragen um die Motivation zu verstehen:
1. **Welchen Ökologischen effekt haben (hohe) Flusswassertemperaturen?**
2. **Für wen ist die Vorhersage wichtige?**
3. **Wann treten sehr hohe Wassertemperaturen auf?**

## 1. Daten

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn import linear_model
import timeit

In [ ]:
# Daten laden und anschauen
data = pd.read_csv("Aschach_data.csv")
data.head()

In [ ]:
# Datum Spalte in index umwandeln und entfernen
data.index = pd.to_datetime(data.date, format = "%Y-%m-%d")
data = data.drop(columns = "date")

In [ ]:
# Wie lang ist die Zeitreihe?
jahre = data.index.year.unique().tolist()
print(f"Die Daten beinhalten die Jahre {', '.join([str(x) for x in [*jahre]])}")

## 2. Explorative data analysis (EDA)

Zuerst ist es wichtig die Daten und die Zusammenhänge zu verstehen. Wa sind das für Daten? Woher kommen sie, was bilden sie ab bzw. welches Gebiet repräsentieren sie? In diesem Abschnitt werden wir Hypothesen generieren, welche uns beim Modellieren helfen werden.

In [ ]:
# Daten-Struktur
data.head()

Beschreibung der Variablen:</br>
**wt**: tägl. mittl. Wassertemperatur in °C</br>
**Q**: tägl. mittl. Abfluss in m³/s</br>
**P**: tägl. Niederschlagssumme in mm</br>
**GL**: tägl. mittl. Kurzwellenstrahlung W/m²</br>
**Ta_min**: tägl. Minimum der Lufttemperatur in °C</br>
**Ta_max**: tägl. Maximum der Lufttemperatur in °C</br>
**Ta**: tägl. mittl. Lufttemperatur in °C

a) wt und Q wurden am Gebietsauslass gemessen. Messpunkte liegen nach rauen Rampen, daher kann eine vollständige Durchmischung angenommen werden.

b) P, Ta_min, Ta_max & Ta sind über das Einzugsgebiet gemittelte Werte basierend auf einem 1x1 km Raster des SPARTACUS Datensatzes (ZAMG, https://www.zamg.ac.at/cms/de/forschung/klima/klimatografien/spartacus)

c) GL sind über das Einzugsgebiet gemittelte Werte basierend auf einem 1x1 km Raster des INCA Datensatzes (ZAMG, https://www.zamg.ac.at/cms/de/forschung/wetter/inca)

### To do:

Schau dir die Daten mal genau an. Code Beispiele dafür findest du unten. </p> Versuche folgende Fragen zu beanworten:
1. Welche Variablen haben potenziell den größten Einfluss auf wt?
2. Reichen die vorhandenen Variablen aus um wt über das Jahr gut vorherzusagen, oder gibt es andere Einflussgrößen die wir berechnen könnten?
3. Brauchen wir Informationen der letzten Tage (lags) um wt vorherzusagen?

### 2.1 EDA Plots

Die folgenden Beispiel Codes können leicht angepast werden. Ihr müsst nur jeweils die Variablen (*var*), und/oder den lag (*lag*) auf die gewünschten Werte ändern, welche von Interesse sind. 

In [ ]:
# Plotte die Zeitreihe(n) einer oder mehrerer Variablen

# Input
var = ["wt"] # Beispiele für meherere Variablen ["Q", "Ta", "wt"]

# Code
plt.figure(figsize=(15,6))
sns.lineplot(data = data[var])
plt.show()

In [ ]:
# Plotte zwei Variablen zusammen in einen Scatterplot

# Inputs
var1 = "wt"
var2 = "GL"

# Code
plt.figure(figsize=(7,7))
sns.scatterplot(x=data[var1], y=data[var2])
plt.show()

In [ ]:
# Plotte wt vs. lagged variables 
# lagged variable = Werte der vorhergehenden Zeitschritte, z.B. Qlag1 ist der Abfluss vom vorherigen Tag

# Inputs
var = "Ta_max"
lag = 8

# Code
lagged_var = pd.concat([pd.Series([np.NaN for i in range(lag+1)]), data[var].iloc[:-(lag+1)]])
lagged_var.index = data[var].index
lagged_var.name = f"{var}_lag{lag}"
lagged_var.head()
plt.figure(figsize=(7,7))
sns.scatterplot(x=data.wt, y=lagged_var)
plt.show()

In [ ]:
# Plotte die Korrelation aller Variablen
# Hier gibt es nichts zum Anpassen --> mach dir ein Bild von den (linearen-) Zusammenhängen zwischen den Variablen
corr = data.corr()
plt.figure(figsize=(9,6))
sns.heatmap(corr, cmap=sns.diverging_palette(230, 20, as_cmap=True), vmin=-1, vmax=1, annot=True)
plt.show()

## 3. Daten pre-processing / feature engineering

Um die relevanten Informationen für die Vorhersage zu verwenden, werden wir neue Features (Variablen) erzeugen!

### 3.1 Lag Variablen

In [ ]:
def create_lags(df, columns, lags):
    """
    Erstellt data frame mit zusätzlich lagged Versionen ausgewählter Spalten des data frames.
    Arguments:
        df: data frame
        columns: Liste mit strings der Spaltennamen
        lags: integer, Anzahl der lags
    Return:
        Einen data frame bestehend aus dem Input data frame mit zusätzlichen Spalten für die lagged Variablen.
    """
    lagged_cols = []
    for var in columns:
        for i in range(lags):
            lagged_var = pd.concat([pd.Series([np.NaN for lag in range(i+1)]), df[var].iloc[:-(i+1)]])
            lagged_var.index = df[var].index
            lagged_var.name = f"{var}_lag{i+1}"
            lagged_cols.append(lagged_var)
    var_lags = pd.concat(lagged_cols, axis=1)
    return pd.concat([df, var_lags], axis=1)

### To do:
Wähle eine Anzahl an lags (lag=1 bedeutet, dass die Variablen des Vortages zusätzlich verwendet werden). 

In [ ]:
# Erzeuge lag Variablen

# Inputs
n_lags = 3

# Code
lagged_cols = [col for col in data.columns if col != "wt"]
data_pp = create_lags(df=data, columns=lagged_cols, lags=n_lags)

In [ ]:
# Zeilen mit NaNs entfernen
data_pp.dropna(inplace=True)

### 3.2 Zeit-Information

**Problem:** Zeit Variablen sind zyklisch --> 1. Tag und 365. Tag im Jahr sind nahezu ident.

Wie können wir aus den zyklischen Variablen der Zeit (Tage des Jahres, Monate, Tage im Monat) eine kontinuierliche Zeit-Variable erzeugen?

&rarr; **Sinus/Consinus Transformation**

![Alt Text](Abb/sine_2.gif)
*Sinus Transformation einer Zyklischen Variable*

![Alt Text](Abb/cosine_2.gif)
*Cosinus Transformation einer Zyklischen Variable*

In [ ]:
data_pp['sin_time'] = np.sin(2*np.pi*data_pp.index.dayofyear/data_pp.index.dayofyear.max())
data_pp['cos_time'] = np.cos(2*np.pi*data_pp.index.dayofyear/data_pp.index.dayofyear.max())

In [ ]:
# Plotte die Zeitreihe(n) einer oder mehrerer Variablen
var = ["sin_time", "cos_time"]
sns.set_style("whitegrid")
plt.figure(figsize=(15,6))
sns.lineplot(data = data_pp[var])
plt.show()

In [ ]:
data_pp.head()

## 4. Data Splits (+Präsentation)

Unabhängige Test Daten und Cross Validierung

In [ ]:
# Train/Test split
test_period = [2013, 2014, 2015]
train_data = data_pp.loc[~data_pp.index.year.isin(test_period)]
test_data = data_pp.loc[data_pp.index.year.isin(test_period)]

print(f"Training samples: {train_data.shape[0]} Tage\nTest samples: {test_data.shape[0]} Tage")

In [ ]:
# Aufspalten in Inputs (x) und Outputs (y)
x_train = train_data.loc[:, train_data.columns != "wt"]
y_train = train_data.wt
x_test = test_data.loc[:, test_data.columns != "wt"]
y_test = test_data.wt

In [ ]:
# Plotte Training & Test Split
sns.set_style("whitegrid")
plt.figure(figsize=(15,7))
plt.title("Mittlere tägliche Wassertemperatur der Aschach")
plt.plot(train_data.index, train_data.wt, color='#379BDB', label='Training')
plt.plot(test_data.index, test_data.wt, color='#fc7d0b', label='Test')
plt.xlabel('')
plt.ylabel('Wassertemperatur [°C]')
plt.legend()
plt.show(block=False)

## 5. Baseline Modell

Warum brauchen wir eine Baseline? --> no free lunch theorem

**Baseline Modell:** Multiple Lineare Regression mit Daten ohne lags.

In [ ]:
# Lineares Modell mit 10-facher CV
start = timeit.default_timer()
x_baseline = x_train.filter(items=data.columns)
baseline_model = linear_model.LinearRegression()
baseline_scores = cross_val_score(baseline_model, x_train.filter(items=data.columns), y_train, 
                                  scoring='neg_root_mean_squared_error', cv=10, n_jobs=-1)
end = timeit.default_timer()

# Model performance
print(f"RMSE: {-np.mean(baseline_scores):.3f} ({np.std(-baseline_scores):.3f})")
#print(end - start)
print(f"Laufzeit: {end - start:.2f} Sekunden")

In [ ]:
# Definiere Gütemaße
def RMSE(x, y):
     return round(((x - y) ** 2).mean() ** .5, 3)
    
def MAE(x, y):
    return round((x-y).abs().mean(), 3)

def print_results(x, y, name):
    print(f"{name} RMSE = {RMSE(x, y)} °C\n{name} MAE = {MAE(x, y)} °C")

Gütemaße:
$$RMSE = \sqrt{\sum_{i=1}^{N}(x_i-y_i)^2}$$
$$MAE = \frac{1}{n} {\sum_{i=1}^{N}|x_i-y_i|}$$

In [ ]:
# Erzeuge Baseline Vorhersage der Test Daten
baseline_model.fit(x_train.filter(items=data.columns), y_train)
baseline_prediction = baseline_model.predict(x_test.filter(items=data.columns))
baseline_prediction_df = pd.DataFrame({"prediction": baseline_prediction})
baseline_prediction_df.index = test_data.index

In [ ]:
# Plotte Baseline Vorhersage
sns.set_style("whitegrid")
plt.figure(figsize=(15,7))
plt.title("Mittlere tägliche Wassertemperatur der Aschach")
plt.plot(test_data.index, test_data.wt, color= '#379BDB', label='Beobachtung')
plt.plot(baseline_prediction_df.index, baseline_prediction_df.prediction, color='#fc7d0b', label='Baseline Modell')
plt.xlabel('')
plt.ylabel('Wassertemperatur [°C]')
plt.legend()
plt.show(block=False)

In [ ]:
print_results(test_data.wt, baseline_prediction, "Baseline")

## 6. Machine Learning Modell (+Präsentation)

### 6.1 Hyperparameter Auswahl

gewähltes Modell: **Random Forest**

### To do:
Probiere unterschiedliche Hyperparameter aus und versuche die beste Kombination zu finden!
Genauere Beschreibung der Parameter von Random Forest findest du auf https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html

Definiere zunächst die Hyperparameter:

In [ ]:
# Hyperparameter
hyperpara = {"n_estimators": 100,    # default=100, Anzahl der Regressionsbäume
             "max_depth": 4,      # default=None, Tiefe des Baums, bei None werden Bäume so lange expandiert bis "min_samples_split" erreicht ist
             "min_samples_split": 2, # default=2, Minimum samples um eine node zu spalten (muss > 1 sein)
             "min_samples_leaf": 1,  # default=1, Mindestanzahl an samples für einen node
             "max_features": 1.0,    # default=1.0, Anzahl an features für split, default=1.0 (100% aller Variablen werden verwendet)
            }

Dann validiere sie mittels Cross Validation:

In [ ]:
# Cross Validierung des Random Forest Modells mit den gewählten Hyperparametern
start = timeit.default_timer() # timer
model = RandomForestRegressor(**hyperpara, 
                              criterion="squared_error",
                              random_state=42) # Modell mit Hyperparameter
n_scores = cross_val_score(model, x_train, y_train, scoring='neg_root_mean_squared_error', cv=10, n_jobs=-1)
end = timeit.default_timer()

# Model performance
print("Random Forest scores")
print(f"RMSE: {-np.mean(n_scores):.3f} ({np.std(-n_scores):.3f}°C), verbesserung zur Baseline: {-np.mean(baseline_scores) + np.mean(n_scores):.3f}°C")
print(f"Laufzeit: {end - start:.2f} Sekunden")

Wir können auch die Vorhersage der Trainingsdaten betrachten nachdem wir mit allen Daten trainiert haben. Glaubst du das wir damit noch zusätzliche Information über die Vorhersagefähigkeit des Modells ermitteln können?

In [ ]:
# Plot training Vorhersage
sns.set_style("whitegrid")
plt.figure(figsize=(18,7))
plt.title("Mittlere tägliche Wassertemperatur der Aschach")

# Training Daten
plt.plot(train_data.index, train_data.wt, color= '#379BDB', label='Beobachtung')

# Random Forest
model.fit(x_train, y_train) # Model fit
train_prediction = model.predict(x_train)
train_prediction_df = pd.DataFrame({"prediction": train_prediction})
train_prediction_df.index = train_data.index
plt.plot(train_prediction_df.index, train_prediction_df.prediction, color='#18d30c', label='Random Forest')
plt.xlabel('')
plt.ylabel('Wassertemperatur [°C]')
plt.legend()
plt.show(block=False)

### 6.2 Modellgüte mit den Test Daten bestimmen

In [ ]:
# Model fit und Vorhersage der Test Daten
model.fit(x_train, y_train)
prediction = model.predict(x_test)
prediction_df = pd.DataFrame({"prediction": prediction})
prediction_df.index = test_data.index
print_results(test_data.wt, prediction, "Random Forest")

In [ ]:
# Verbesserung im Vergleich zum Baseline Modell
rmse_red = RMSE(test_data.wt, baseline_prediction) - RMSE(test_data.wt, prediction)
mae_red = MAE(test_data.wt, baseline_prediction) - MAE(test_data.wt, prediction)
print(f"RMSE Verbesserung im Vergleich zum Baseline Modell: {rmse_red:.3f} °C")
print(f"MSE Verbesserung im Vergleich zum Baseline Modell: {mae_red:.3f} °C")

In [ ]:
# Plot der Test Vorhersagen für ein Testjahr: Beobachtungen, Baseline & Random Forest
jahr = 2014
sns.set_style("whitegrid")
plt.figure(figsize=(18,7))
plt.title("Mittlere tägliche Wassertemperatur der Aschach")

# Test Daten
test_plot = test_data[test_data.index.year == jahr]
plt.plot(test_plot.index, test_plot.wt, color= '#379BDB', label='Beobachtung')

# Baseline Modell
base_plot = baseline_prediction_df[baseline_prediction_df.index.year == jahr]
plt.plot(base_plot.index, base_plot.prediction, color='#fc7d0b', label='Baseline Modell')

# Random Forest
pred_plot = prediction_df[prediction_df.index.year == jahr]
plt.plot(pred_plot.index, pred_plot.prediction, color='#18d30c', label='Random Forest')
plt.xlabel('')
plt.ylabel('Wassertemperatur [°C]')
plt.legend()
plt.show(block=False)

#### Die besten Ergebisse:

Diskussion + vergleichen der Ergebnisse


## 7. Zweites ML Modell

Wähle eines zweites ML Modell und wende es auf die Daten an. Alles ist erlaubt. Finde anhand der Beschreibungen ein passendes Modell für dieses (regressions Problem). Nutze dazu die Liste an Modellen in scikit-learn:</p>
https://scikit-learn.org/stable/supervised_learning.html



Als Hilfestellung findest du nachfolgend ein Template:

In [ ]:
# Cross Validierung eines Modells 
start = timeit.default_timer() # timer

# ----------------------------------------
# Hier muss das Modell (und gegebenenfalls die Hyperparameter) definiert werden
model2 = ...
# ----------------------------------------

# Cross Validierung des Modells
n_scores = cross_val_score(model2, x_train, y_train, scoring='neg_root_mean_squared_error', cv=10, n_jobs=-1)
end = timeit.default_timer()

# Model performance
print(f"RMSE: {-np.mean(n_scores):.3f} ({np.std(-n_scores):.3f}), verbesserung zur Baseline: {-np.mean(baseline_scores) + np.mean(n_scores):.3f}°C"))
print(f"Laufzeit: {end - start:.2f} Sekunden")

In [ ]:
# Modell fit und Vorhersage der Test Daten
model2.fit(x_train, y_train) # Model fit
prediction2 = model2.predict(x_test)
prediction2_df = pd.DataFrame({"prediction": prediction2})
prediction2_df.index = test_data.index
print_results(test_data.wt, prediction2, "Alternativmodell")

In [ ]:
# Verbesserung im Vergleich zum Baseline Modell
rmse_red = RMSE(test_data.wt, baseline_prediction) - RMSE(test_data.wt, prediction2)
mae_red = MAE(test_data.wt, baseline_prediction) - MAE(test_data.wt, prediction2)
print(f"RMSE Verbesserung im Vergleich zur Baseline: {rmse_red:.3f} °C")
print(f"MSE Verbesserung im Vergleich zur Baseline: {mae_red:.3f} °C")

In [ ]:
# Verbesserung im Vergleich zum Random Forest Modell
rmse_red = RMSE(test_data.wt, prediction) - RMSE(test_data.wt, prediction2)
mae_red = MAE(test_data.wt, prediction) - MAE(test_data.wt, prediction2)
print(f"RMSE Verbesserung im Vergleich zum Random Forest: {rmse_red:.3f} °C")
print(f"MSE Verbesserung im Vergleich zum Random Forest: {mae_red:.3f} °C")

# 8. Evaluierung der Ergebnisse

Gruppendiskussion:
1. Welches Modell und welche Hyperparameter liefern die besten Ergebnisse?
2. Wie könnten wir unsere Modellierung verbessern?


Hier findest du die Ergebnisse unserer Publikation mit verschiedensten Modellen und Hyperparameteroptimierung:
![Übersicht Aschach Ergebnisse (Feigl et al., 2021)](Abb/paper_results.png)
*Übersicht Ergebnisse Feigl et al. (2021)*